This notebook contains the implementation of the Research Paper:  A deep learning method DCWR with HANet for stock market prediction using news articles

In [1]:
import tensorflow as tf

# Load SavedModel format of ELMO model
elmo1 = tf.saved_model.load(r'E:\Semesters\Fyp prepation\ELMO')

In [1]:
import pickle
with open(r'AAPL_articals_list','rb') as f:
    list_art = pickle.load(f)

In [2]:
sen_len=[]
for art in list_art:
    sen_len.append(len(art))
max(sen_len)

238

In [110]:
lengt =[]
for i in list_art:
    lengt.append(len(i))
    
sum = sum(lengt)
avg = sum / len(list_art)
avg



28.3114672008012

In [90]:
len(list_art[93])

35

In [4]:
embeddings = elmo.signatures["default"](tf.constant(list_art[:1]))["elmo"]

NameError: name 'elmo' is not defined

In [12]:
import time
class Embeddings:
    
    def get_elmo_embeddings(elmo , Progress, artical_list : list):
        embeddings_list =[]
        total_indexes = len(artical_list)
        progress = Progress()   
        
        for count, artical in enumerate(artical_list):
            start_time = time.time()
            embeddings_list.append(elmo.signatures["default"](tf.constant(artical))["elmo"])
            time_taken = time.time() - start_time
            progress.print_progress(count,total_indexes,time_taken)

            
        return embeddings_list

Got the Word Embeddings of news Atricals in the cell below

In [128]:
from FunctionProgress1 import Progress


X = Embeddings.get_elmo_embeddings(elmo1, Progress, list_art)

In [129]:
embeddings = X

In [93]:
import numpy as np 
max_length = max(embeddings.shape[1] for embeddings in X)

# Pad the embeddings with zeros to make them all have the same shape
padded_embeddings = []
for embeddings in X:
    num_sentences, original_length, embedding_size = embeddings.shape
    padding_size = max_length - original_length
    padding = np.zeros((num_sentences, padding_size, embedding_size))
    padded_embeddings.append(np.concatenate([embeddings, padding], axis=1))
embeddings = np.concatenate(padded_embeddings)

In [94]:
num_sentences, max_length, embedding_size = embeddings.shape
embeddings = embeddings.reshape(num_sentences, max_length * embedding_size)



<h3>implementing the ICA to the word Embeddings in the cell below </h3>

In [ ]:
import numpy as np
from sklearn.decomposition import FastICA
from FunctionProgress1 import Progress


# Assuming we have obtained the ELMO embeddings in a list of numpy arrays `embeddings` of shape (n_words, embedding_size)

# Apply ICA on each embedding in the list to extract independent features
ica = FastICA(n_components=31, random_state=0)
ica_embeddings = []

progress = Progress()
total_indexes = len(embeddings)

for count ,embedding in enumerate(embeddings):
    start_time = time.time()
    
    # Reshape the embedding to 2D array of shape (n_words, embedding_size)
    X = np.reshape(embedding, (-1, embedding.shape[-1]))
    # Apply ICA on the flattened embedding to extract independent features
    X_ica = ica.fit_transform(X)
    # Reshape the extracted features back to 2D array of shape (n_words, n_components)
    ica_embeddings.append(X_ica)
    
    time_taken = time.time() - start_time
    progress.print_progress(count,total_indexes,time_taken)
    
# Convert the list of ICA embeddings to numpy array
ica_embeddings = np.array(ica_embeddings) 


In [131]:


with open("ICA_embeddings", 'wb') as f:
    pickle.dump(ica_embeddings,f)

In [3]:
import pickle

with open("ICA_embeddings", 'rb') as f:
   ica_embeddings=pickle.load(f)

In [4]:
import pandas as pd
df2 = pd.read_csv('E:\Semesters\Fyp prepation\Data_preprocessing\Data_searching\Apple_dataset.csv', index_col= False)
df2.dropna(inplace=True)
df2.reset_index(drop=True, inplace=True)
dates = pd.to_datetime(df2['Data_Time']).dt.date

date_embedding_mapping = dict(zip(dates, ica_embeddings))


In [5]:
df_AAPL_STOCK = pd.read_csv(r'E:\Semesters\Fyp prepation\Dataset\AAPL_Stock_Dataset/AAPL.csv')
df_AAPL_STOCK['Date'] = pd.to_datetime(df_AAPL_STOCK['Date']).dt.date
df_AAPL_STOCK = df_AAPL_STOCK[['Date','Close']]
df_AAPL_STOCK

,Date,Close
0,2005-01-03,1.130179
1,2005-01-04,1.141786
2,2005-01-05,1.151786
3,2005-01-06,1.152679
4,2005-01-07,1.236607
...,...,...
2260,2013-12-24,20.273930
2261,2013-12-26,20.139286
2262,2013-12-27,20.003214
2263,2013-12-30,19.804285


In [6]:
df_stock_price_embeddings = pd.merge(df_AAPL_STOCK, pd.DataFrame(list(date_embedding_mapping.items()),columns=['Date', 'embeddings']), on='Date')

In [7]:
df_stock_price_embeddings

,Date,Close,embeddings
0,2007-03-28,3.330000,"[[-0.019289728, -0.025915716, -0.010203429, 0...."
1,2007-07-20,5.133929,"[[-0.0015902107, 0.0024687697, -0.0030268093, ..."
2,2010-01-28,7.117500,"[[-0.0056771897, -0.0062624235, -0.0047936994,..."
3,2010-02-26,7.307857,"[[0.036892436, 0.0042556855, 0.088577226, -0.0..."
4,2010-04-23,9.672500,"[[-0.04697889, -0.04293068, -0.018973464, -0.0..."
...,...,...,...
544,2013-11-19,18.555357,"[[0.00060118176, -0.00432523, -0.012103775, 0...."
545,2013-11-20,18.392857,"[[-0.0033492912, 0.0006496103, -0.004905187, 0..."
546,2013-11-21,18.612143,"[[-0.06283923, 0.06635708, 0.05533373, 0.02343..."
547,2013-11-22,18.564285,"[[-0.030148793, -0.04174973, 0.06181645, -0.03..."


In [8]:
import pandas as pd
from datetime import timedelta

def generate_time_series(embeddings_df, stock_prices_df,lookahead_days):
    lookahead_columns = ['Close+' +str(i) for i in range(1, lookahead_days+1)]
    
    new_df = pd.DataFrame(columns=['embeddings', 'Close']+lookahead_columns)
    
    # stock_prices_df['Date'] = pd.to_datetime(stock_prices_df['Date'])
    # stock_prices_df = stock_prices_df.sort_values(by='Date')
    
    for index, row in embeddings_df.iterrows():
        date = row['Date']
        embedding = row['embeddings']
        
        # Find the stock price of the date of the embedding
        stock_price = stock_prices_df[stock_prices_df['Date'] == date]['Close'].values[0]
        
        lookahead_dates = [date + timedelta(days=i) for i in range(1, 8)]
        lookahead_prices = []
        
        for lookahead_date in lookahead_dates:
            if lookahead_date in stock_prices_df['Date'].values:
                lookahead_price = stock_prices_df[stock_prices_df['Date'] == lookahead_date]['Close'].values[0]
            else:
                lookahead_price = lookahead_price  # Use the last available price
            lookahead_prices.append(lookahead_price)
        
        # Create the dictionary of column values
        columns_values = {'embeddings': embedding, 'Close': stock_price}
        columns_values.update(zip(lookahead_columns, lookahead_prices))        
        
        # Append the data to the new dataframe
        new_df = new_df.append(columns_values, ignore_index=True)
    
    return new_df


In [9]:
df_time_series_emb=  generate_time_series(df_stock_price_embeddings, df_AAPL_STOCK, 5)
df_time_series_emb

,embeddings,Close,Close+1,Close+2,Close+3,Close+4,Close+5
0,"[[-0.019289728, -0.025915716, -0.010203429, 0....",3.330000,3.348214,3.318214,3.318214,3.318214,3.344643
1,"[[-0.0015902107, 0.0024687697, -0.0030268093, ...",5.133929,3.366786,3.366786,5.132143,4.817500,4.902143
2,"[[-0.0056771897, -0.0062624235, -0.0047936994,...",7.117500,6.859286,6.859286,6.859286,6.954643,6.995000
3,"[[0.036892436, 0.0042556855, 0.088577226, -0.0...",7.307857,6.858929,6.858929,7.463929,7.458929,7.476071
4,"[[-0.04697889, -0.04293068, -0.018973464, -0.0...",9.672500,7.819643,7.819643,9.625000,9.358571,9.342857
...,...,...,...,...,...,...,...
544,"[[0.00060118176, -0.00432523, -0.012103775, 0....",18.555357,18.392857,18.612143,18.564285,18.564285,18.564285
545,"[[-0.0033492912, 0.0006496103, -0.004905187, 0...",18.392857,18.612143,18.564285,18.564285,18.564285,18.705000
546,"[[-0.06283923, 0.06635708, 0.05533373, 0.02343...",18.612143,18.564285,18.564285,18.564285,18.705000,19.049999
547,"[[-0.030148793, -0.04174973, 0.06181645, -0.03...",18.564285,19.498571,19.498571,18.705000,19.049999,19.498571


<h4>In the cells below we are trying to construct and train the HANet Network, but we failed to do so.
We took multiple approach to construct the network, so the below cells are all messed up.

We got the word embeddings, we applied the ICA on the word Embeddings, then we got the time series data of ica applied embeddings and stock prices.

The only thing left is the HANet network construction and traning.</h4>

In [11]:
import tensorflow as tf

In [12]:
class AdditiveAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(AdditiveAttention, self).__init__(**kwargs)
        self.W_q = self.add_weight(shape=(units, units), initializer='uniform', trainable=True)
        self.W_k = self.add_weight(shape=(units, units), initializer='uniform', trainable=True)
        self.V = self.add_weight(shape=(units, 1), initializer='uniform', trainable=True)

    def call(self, inputs):
        q = tf.matmul(inputs, self.W_q)
        k = tf.matmul(inputs, self.W_k)
        attn_scores = tf.matmul(tf.nn.tanh(q + k), self.V)
        attn_scores = tf.nn.softmax(attn_scores, axis=1)
        output = tf.reduce_sum(inputs * attn_scores, axis=1)
        return output


In [13]:
from tensorflow.python.framework.ops import disable_eager_execution

class HANetModel(tf.keras.Model):
    def __init__(self, sentence_length, embedding_dim, hidden_units, num_future_days, attention_units):
        # disable_eager_execution()
        tf.compat.v1.disable_eager_execution()

        super(HANetModel, self).__init__()

        # Sentence-level input and attention layers
        self.sentence_input = tf.keras.layers.Input(shape=(sentence_length, embedding_dim))
        self.sentence_lstm = tf.keras.layers.LSTM(units=hidden_units, return_sequences=True)(self.sentence_input)
        self.sentence_attention = AdditiveAttention(units=attention_units)

        # Flatten layer for stock price prediction
        self.flatten = tf.keras.layers.Flatten()

        # Dense layers for stock price prediction
        self.forecast = tf.keras.layers.Dense(units=num_future_days)

    def call(self, inputs):
        sentence_inputs = inputs

        # Apply the sentence attention layer
        sentence_attention = self.sentence_attention(self.sentence_lstm)

        # Flatten and Dense layer for stock price forecasting
        flatten = self.flatten(sentence_attention)
        forecast = self.forecast(flatten)

        return forecast

In [14]:
class MeanSquaredError(tf.keras.layers.Layer):
    def call(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_true - y_pred))

In [29]:
# Assuming you have a DataFrame 'df_time_series_emb' with a column 'embeddings' containing word embeddings (as numpy arrays)

# Step 1: Convert the 'embeddings' column to a list of numpy arrays
sequences = df_time_series_emb['embeddings'].tolist()

# Step 2: Find vocab_size, sentence_length, and word_length
vocab_size = len(sequences)
sentence_length = sequences[0].shape[0]  # Assuming the word embeddings are 2D arrays with consistent dimensions
word_length = sequences[0].shape[1]
embedding_dim = sequences[0].shape[1]  # Same as word_length
hidden_units = embedding_dim
num_future_days =5
# Now you have the values for YOUR_VOCAB_SIZE, YOUR_MAX_SENTENCE_LENGTH, YOUR_MAX_WORD_LENGTH, and YOUR_EMBEDDING_DIMENSION


In [157]:
# Create and compile the model
model = HANetModel(sentence_length, embedding_dim, hidden_units, num_future_days)
model.compile(optimizer='adam', loss=MeanSquaredError())

In [109]:
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences



sentence_embeddings_data =df_time_series_emb['embeddings'].tolist()
max_sentence_length = max(len(seq) for seq in sentence_embeddings_data)

sentence_embeddings_data = pad_sequences(sentence_embeddings_data, maxlen=max_sentence_length, dtype=np.float32)

sentence_embeddings_data = np.array(sentence_embeddings_data, dtype=np.float32)

# current_stock_price = np.array(df_time_series_emb['Close'])
future_stock_prices = np.array(df_time_series_emb.iloc[:, -6:])  # Keep only the last 5 days' stock prices

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings_data, future_stock_prices, test_size=0.2, random_state=42)


In [17]:
import tensorflow as tf

class ReshapeLayer(tf.keras.layers.Layer):
    def __init__(self, target_shape, **kwargs):
        super(ReshapeLayer, self).__init__(**kwargs)
        self.target_shape = target_shape

    def call(self, inputs):
        return tf.reshape(inputs, self.target_shape)


In [26]:
import tensorflow as tf

# X_train_flat = np.array([np.array(x) for x in X_train])


# Convert NumPy arrays to TensorFlow tensors
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)


# Reshape y_train_tf and y_test_tf to have shape (None, num_future_days)
reshape_layer = ReshapeLayer(target_shape=(-1, num_future_days))
y_train_tf = reshape_layer(y_train_tf)
y_test_tf = reshape_layer(y_test_tf)
num_samples = X_train_tf.shape[0]
steps_per_epoch = num_samples


hanet_model = HANetModel(sentence_length=11484, embedding_dim=31, hidden_units=64, num_future_days=5, attention_units= 64)
hanet_model.compile(optimizer='adam', loss='mse')

# Create a TensorFlow session and initialize variables
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

# Now you can use the TensorFlow tensors in the model.fit() function

# hanet_model.fit(X_train_tf, y_train_tf, epochs=10, batch_size=32, steps_per_epoch=steps_per_epoch,validation_data=(X_test_tf, y_test_tf), session=sess)


# # Make predictions
# predictions = hanet_model.predict(X_test_tf, session=sess)
# print(predictions)  # This will give you the predicted future stock prices

In [27]:
y_train_tf.shape

TensorShape([439, 5])

In [162]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
    # Compile the model
    model.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['mean_squared_error'])

    # Convert data to tensors
    X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
    X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)
    
    # Reshape y_train_tf and y_test_tf to have shape (None, num_future_days)
    y_train_tf = tf.reshape(y_train_tf, (-1, num_future_days))
    y_test_tf = tf.reshape(y_test_tf, (-1, num_future_days))
    # Train the model
    model.fit(X_train_tf, y_train_tf, epochs=epochs, batch_size=batch_size, validation_data=(X_test_tf, y_test_tf))


In [126]:
model = HANetModel(sentence_length, embedding_dim, hidden_units, num_future_days)

# Train the model
train_model(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32)

Epoch 1/10


TypeError: in user code:

    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 239, in __call__
        self._loss_metric.update_state(
    File "c:\Users\Abdul\anacon

In [83]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_q = self.add_weight(name='W_q',
                                   shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform',
                                   trainable=True)
        self.W_k = self.add_weight(name='W_k',
                                   shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform',
                                   trainable=True)
        self.W_v = self.add_weight(name='W_v',
                                   shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform',
                                   trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        q = tf.matmul(inputs, self.W_q)
        k = tf.matmul(inputs, self.W_k)
        v = tf.matmul(inputs, self.W_v)

        attn_scores = tf.matmul(q, k, transpose_b=True)
        attn_scores = tf.nn.softmax(attn_scores)
        output = tf.matmul(attn_scores, v)

        return output


In [84]:
class HANetModel(tf.keras.Model):
    def __init__(self, sentence_length, embedding_dim, hidden_units, num_future_days):
        super(HANetModel, self).__init__()

        # Sentence-level input and attention layers
        self.sentence_input = tf.keras.layers.Input(shape=(sentence_length, embedding_dim))
        self.sentence_lstm = tf.keras.layers.LSTM(units=hidden_units, return_sequences=True)(self.sentence_input)
        self.sentence_attention = AttentionLayer()(self.sentence_lstm)

        # Flatten layer for stock price prediction
        self.flatten = tf.keras.layers.Flatten()

        # Dense layers for stock price prediction
        self.forecast = tf.keras.layers.Dense(units=num_future_days)

    def call(self, inputs):
        sentence_inputs = inputs

        # Apply the sentence attention layer
        sentence_attention = self.sentence_attention(sentence_inputs)

        # Flatten and Dense layer for stock price forecasting
        flatten = self.flatten(sentence_attention)
        forecast = self.forecast(flatten)

        return forecast


In [85]:
# Assuming you have defined all the necessary variables such as sentence_length, embedding_dim, hidden_units, and num_future_days.

# Create an instance of the HANetModel
model = HANetModel(sentence_length, embedding_dim, hidden_units, num_future_days)

# Compile the model with an appropriate optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_function = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss_function)


In [152]:
# Assuming you have X_train and y_train as your training data, and X_test and y_test as your test data.

# Train the model with your training data
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10


ValueError: in user code:

    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Abdul\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 200, in assert_input_compatibility
        raise ValueError(f'Layer "{layer_name}" expe

In [90]:
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate, RepeatVector, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np

# Assuming you have the sequences of word embeddings and stock data features
# sequences: List of word embeddings for news articles
# stock_data: 2D array containing historical stock data features
# labels: 1D array containing the corresponding stock prices for the future days

# Step 1: Preprocess the word embeddings (sequences) and stock data
# sequences = ...
# stock_data = ...
# labels = ...

# Step 2: Find vocab_size, sentence_length, and word_length
# vocab_size = len(sequences)
# sentence_length = sequences[0].shape[0]  # Assuming the word embeddings are 2D arrays with consistent dimensions
# word_length = sequences[0].shape[1]
# embedding_dim = sequences[0].shape[1]  # Same as word_length
# hidden_units = embedding_dim
# num_future_days = 5

attention_units = 64  # Replace with the attention units
num_stock_features = X_train.shape[1]  # Replace with the actual number of stock data features

# Step 3: Define the HANetModel
class HANetModel(tf.keras.Model):
    def __init__(self, sentence_length, word_length, vocab_size, embedding_dim, hidden_units, num_future_days, attention_units, num_stock_features):
        tf.config.experimental_run_functions_eagerly(True)
        # tf.compat.v1.disable_eager_execution()


        super(HANetModel, self).__init__()

        # Sentence-level input and attention layers
        self.sentence_input = Input(shape=(sentence_length, word_length))
        self.sentence_lstm = LSTM(units=hidden_units, return_sequences=True)

        # Stock data input and attention layers
        self.stock_input = Input(shape=(num_stock_features,))
        self.stock_attention = Dense(units=attention_units, activation='tanh')
        self.stock_attention_repeated = RepeatVector(sentence_length)

        # Attention fusion
        self.attention_fusion = Concatenate(axis=-1)

        # Flatten and Dense layer for stock price forecasting
        self.flatten = Flatten()
        self.forecast = Dense(units=num_future_days)

    def call(self, inputs):
        sentence_inputs, stock_inputs = inputs

        # Apply the sentence LSTM
        sentence_lstm = self.sentence_lstm(self.sentence_input)

        # Apply the stock attention layer
        stock_attention = self.stock_attention(self.stock_input)
        stock_attention_repeated = self.stock_attention_repeated(stock_attention)

        # Attention fusion
        attention_fusion = self.attention_fusion([sentence_lstm, stock_attention_repeated])

        # Flatten and Dense layer for stock price forecasting
        flatten = self.flatten(attention_fusion)
        forecast = self.forecast(flatten)

        return forecast
# Step 4: Convert NumPy arrays to TensorFlow tensors
X_train_seq = np.array(sequences)  # Assuming you have the training sequences for word embeddings
X_train_stock = np.array(X_train)  # Assuming you have the training stock data features
y_train = np.array(y_train)  # Assuming you have the training stock prices for the future days


attention_units = 64  # Replace with the attention units
num_stock_features = X_train_stock.shape[1]  # R

# Step 5: Instantiate the model
# hanet_model = HANetModel(sentence_length, word_length, vocab_size, embedding_dim, hidden_units, num_future_days, attention_units, num_stock_features)

# # Step 6: Compile and train the model
# hanet_model.compile(optimizer='adam', loss='mean_squared_error') 
# hanet_model.fit(x=[X_train_seq, X_train_stock], y=y_train, epochs=10, batch_size=32)


<ipython-input-90-8735d9a5aa78>:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_seq = np.array(sequences)  # Assuming you have the training sequences for word embeddings


In [76]:
import numpy as np
import pandas as pd

# Assuming you have your DataFrame named df_time_series_emb
# Extract the embeddings and stock data columns
embeddings_column =  df_time_series_emb['embeddings']

stock_prices =  df_time_series_emb[['Close','Close+1', 'Close+2', 'Close+3', 'Close+4', 'Close+5']]

# Convert the embeddings column to a numpy array
X_train_seq = np.array(embeddings_column.tolist())

# Convert the stock data columns to a numpy array
# X_train_stock = np.array(future_stock_prices)
y_train = np.array(stock_prices)


# Print the shapes to verify
print("Shape of X_train_seq:", X_train_seq.shape)  # Should be (number of samples, sentence_length, word_length)
print("Shape of X_train_stock:", y_train.shape)  # Should be (number of samples, number of stock data features)

len(embeddings_column)

Shape of X_train_seq: (549,)
Shape of X_train_stock: (549, 6)


<ipython-input-76-2941b7049e5d>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_seq = np.array(embeddings_column.tolist())


549

In [110]:
# Define the HANet model
hanet_model = HANetModel(sentence_length, word_length, vocab_size, embedding_dim, hidden_units, num_future_days, attention_units, num_stock_features)

# Compile the model
hanet_model.compile(optimizer='adam', loss='mse')


# Train the model
# tf.config.experimental_run_functions_eagerly(True)
@tf.function
def train():
    hanet_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


train()



TypeError: You are attempting to use Python control flow in a layer that was not declared to be dynamic. Pass `dynamic=True` to the class constructor.
Encountered error:
"""
in user code:

    File "<ipython-input-85-e5a7cd413566>", line 53, in call  *
        sentence_inputs, stock_inputs = inputs

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

"""

In [106]:
X_train_seq = np.concatenate(X_train_seq, axis=0)  # Convert to a single 3D array


In [108]:
X_train.shape

(439, 11484, 31)

In [102]:
y_train.shape

TensorShape([549, 6])

In [ ]:
# Stock Data Fusion:

# Concatenate the sentence-level representations obtained from the attention mechanism with the corresponding stock data features. This creates a joint representation of the news articles and stock data for each time step.
# Forecasting:

# Apply fully connected layers on top of the fused representations to perform stock price prediction. The output layer will produce the predicted stock prices for the next time steps.